# Apache Spark's Structured APIs



## Prepare environment
First, we are going to prepare the environment for running PySaprk in the Google Collab Machine (if you work directly in your computer, and you want to prepare it, read and follow champter 2 instructions)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python /content/drive/MyDrive/UDL/2024/install_pyspark.py

## Start working with Spark
Now we now and understand how Spark appeared in our lives and more or less how it works (and you know, it's amazing 🤭), we can start to work with it.
As you now, the SparkSession is the way programmers "talk" with Spark. So, we need to inicialize that.

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession
 .builder
 .appName("example")
 .getOrCreate())

## Example of working with RDDs
But remember, since Spark 2.X we have Structured Data APIs and WE 🧡 DF

In [ ]:
from pyspark import SparkContext
# Create an RDD of tuples (name, age)
dataRDD = spark.sparkContext.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)])
# Use map and reduceByKey transformations with their lambda
# expressions to aggregate and then compute average
agesRDD = (dataRDD
.map(lambda x: (x[0], (x[1], 1)))
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
.map(lambda x: (x[0], x[1][0]/x[1][1])))
agesRDD.collect()

## Example of working with DFs
Yep, as you can see it's easier, clearlier, more bueatiful... (it looks like pandas, isn't it?)

In [ ]:
from pyspark.sql.functions import avg

data_df = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)], ["name", "age"])
# Group the same names together, aggregate their ages, and compute an average
avg_df = data_df.groupBy("name").agg(avg("age"))
# Show the results of the final execution
avg_df.show()

## Define schemas
There are two ways to defines them:

### Define the schema programatically

In [ ]:
from pyspark.sql.types import *
schema_programatically = StructType([StructField("Id", IntegerType(), False),
  StructField("First", StringType(), False),
  StructField("Last", StringType(), False),
  StructField("Url", StringType(), False),
  StructField("Published", StringType(), False),
  StructField("Hints", IntegerType(), False),
  StructField("Campaigns", ArrayType(StringType()), False),
 ])

### Define the schema using DDL

In [ ]:
schema_ddl = "`Id` INT, `First` STRING, `Last` STRING, `Url` STRING, `Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>"



### Example creating data with both

In [ ]:
#create our data
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
       [2, "Brooke","Wenig","https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3","6/7/2019",7659, ["web", "twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das","https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]
      ]

In [ ]:
# create a DataFrame using the schema built programatically
blogs_df = spark.createDataFrame(data, schema_programatically)
blogs_df.show()

In [ ]:
# create a DataFrame using the schema built programatically
blogs_df = spark.createDataFrame(data, schema_ddl)
blogs_df.show(truncate=False)

## Exercice 1
As you can see, the previous schema are not exactly the same. In the programatically way, we can specify not nulleable values. Any idea about how to do the same with DDL?

In [ ]:
#pass

### Get Schema from DF

In [ ]:
blogs_df.schema

### Read data from json
Download data from https://github.com/databricks/LearningSparkV2/blob/master/chapter3/scala/data/blogs.json

In [ ]:
import wget
json_url = "https://raw.githubusercontent.com/databricks/LearningSparkV2/master/chapter3/scala/data/blogs.json"
wget.download(json_url)

In [ ]:
blogs_df = spark.read.json('blogs.json')

In [ ]:
blogs_df = spark.read.schema(schema_ddl).json('blogs.json')

*Return to slides*

## Columns and Expressions

### List all columns of DF

In [ ]:
blogs_df.columns

### Access to particular columns with col function

In [ ]:
blogs_df["Id"]

### Different ways of computing values

In [ ]:
from pyspark.sql.functions import expr
blogs_df.select(expr("Hits * 2")).show()

In [ ]:
blogs_df.selectExpr("Hits * 2").show()

In [ ]:
from pyspark.sql.functions import col
blogs_df.select(col("Hits") * 2).show()

### Create new columns

In [ ]:
blogs_df.withColumn("Big Hitters", col("Hits")>10000).show()

In [ ]:
# big_hitters_df = blogs_df.withColumn("Big Hitters", col("Hits")>10000)

For our mental health: import pyspark.sql.functions as F

In [ ]:
import pyspark.sql.functions as F
blogs_df \
  .withColumn("CompleteName", F.concat(F.col("First"), F.lit(""), F.col("Last"))).show()

### Select (project) some columns

In [ ]:
blogs_df.select(F.col("Hits")).show()

In [ ]:
blogs_df.select("Hits").show()

In [ ]:
blogs_df.select(F.col("Hits"), F.col("Id")).show()

In [ ]:
blogs_df.select(["Hits", "Id"]).show()

### Sort values

In [ ]:
blogs_df.sort(col("Hits"),ascending=False).show()

*Return to slides*

## Rows

We can get all the df's records as a list of :class:`Row`

In [ ]:
blogs_df.collect()

We also can create Row objects

In [ ]:
from pyspark.sql import Row
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", ["twitter"])

We can access elements in row by position:

In [ ]:
blog_row[0]

or by column name:

In [ ]:
blog_row["Id"]

The problem here, is that Spark doen't know the columns names... to fix we could create it like:

In [ ]:
blog_named_row = Row(Id=6, First="Reynold", Last="Xin", Url="https://tinyurl.6", Hits=255568, Published="3/2/2015", Campaigns=["twitter"])

In [ ]:
blog_named_row["Id"]

We can create a DF from a list of  Raws

In [ ]:
df_from_named_raws = spark.createDataFrame([blog_named_row])

In [ ]:
df_from_raws = spark.createDataFrame([blog_row], ['Id', 'First', 'Last', 'Url', 'Hits', 'Published', 'Campaigns'])

## Write results

In [ ]:
parquet_path= "blogs_df.parquet"
blogs_df.write.parquet(parquet_path)

In [ ]:
json_path= "blogs_df.json"
blogs_df.write.json(json_path)

Return to slides

## Filters

In [ ]:
blogs_df.filter(F.col("Hits") > 10000).show()

In [ ]:
blogs_df.where(F.col("Hits") > 10000).where(F.array_contains(F.col("Campaigns"),"FB")).show()

In [ ]:
wget.download('https://github.com/databricks/LearningSparkV2/raw/master/chapter3/data/sf-fire-calls.csv')
fire_df = spark.read.csv('sf-fire-calls.csv', header = True)

In [ ]:
fire_df.show()

In [ ]:
medical_inc_df = fire_df\
  .select("IncidentNumber", "AvailableDtTm", "CallType")\
  .where(F.col("callType") != "Medical Incident")

In [ ]:
medical_inc_df.show(5, False)

### Aggregations
Let's imagine, we want to count different kinds of calls

In [ ]:
fire_df\
  .select("CallType")\
  .where(F.col("CallType").isNotNull())\
  .agg(F.count_distinct("CallType").alias("diff_types"))\
  .show()

And list them

In [ ]:
fire_df\
  .select("CallType")\
  .where(F.col("CallType").isNotNull())\
  .distinct()\
  .show(30, truncate=False)

 ### Renaming, adding and dropping columns

In [ ]:
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")

In [ ]:
new_fire_df.printSchema()

Now, usual data processing: change dates to timestamps:

In [ ]:
fire_ts_df = (new_fire_df
 .withColumn("IncidentDate", F.to_timestamp(col("CallDate"), "MM/dd/yyyy"))
 .drop("CallDate")
 .withColumn("OnWatchDate", F.to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
 .drop("WatchDate")
 .withColumn("AvailableDtTS", F.to_timestamp(col("AvailableDtTm"),
 "MM/dd/yyyy hh:mm:ss a"))
 .drop("AvailableDtTm"))

In [ ]:
fire_ts_df.show()

### Aggregations

 + What are the most common types of fire calls?

In [ ]:
(fire_ts_df
 .select("CallType")
 .where(col("CallType").isNotNull())
 .groupBy("CallType")
 .count() # it's a kind of aggregation
 .orderBy("count", ascending=False)
 .show(n=10, truncate=False))

In [ ]:
(fire_ts_df
 .select(F.sum("NumAlarms"),
         F.avg("ResponseDelayedinMins"),
         F.min("ResponseDelayedinMins"),
         F.max("ResponseDelayedinMins"))
 .show())


+ What were all the different types of fire calls in 2018?
+ What months within the year 2018 saw the highest number of fire calls?
+ Which neighborhood in San Francisco generated the most fire calls in 2018?
+ Which neighborhoods had the worst response times to fire calls in 2018?
+ Which week in the year in 2018 had the most fire calls?
+ Is there a correlation between neighborhood, zip code, and number of fire calls?
+ How can we use Parquet files or SQL tables to store this data and read it back?
